In [2]:
import cv2
import numpy as np
import math



black_range = np.array([(0, 0, 170), (360, 50, 255)])
capture = cv2.VideoCapture(0)


def maskBlack(img):
    img = cv2.bitwise_not(img) #img 색 반전
    mask = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #bgr에서 hsv로 변환
    mask = cv2.inRange(mask, black_range[0], black_range[1]) #black_range안에 있는것만 걸러낸다고 지정
    img = cv2.bitwise_and(img, img, mask=mask) #채로 걸러낸다.
    return img



def chkArrow(rect):
    return rect['w'] > rect['h']*1.5


def chkSize(rect):
    return rect['w']*rect['h'] > 200


def extractRect(img, *funcs):
    rects = []
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(
        img,
        mode=cv2.RETR_EXTERNAL,
        method=cv2.CHAIN_APPROX_SIMPLE
    ) #coutours에 Numpy 구조의 배열로 검출된 윤곽선의 지점들이 담겨있습니다.
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)#boundingRect는 직사각형 그려주는 함수
        rects.append({
            'x': x,
            'y': y,
            'w': w,
            'h': h,
            'cx': x + (w / 2),
            'cy': y + (h / 2)
        })
        #cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
    for func in funcs:
        temp = []
        for rect in rects:
            if func(rect):
                temp.append(rect)
        rects = temp.copy()
    return rects


def detectArrow(img):
    arrows = []
    img = maskBlack(img)
    rects = extractRect(img, chkArrow, chkSize)
    for rect in rects:
        img_cropped = cv2.getRectSubPix(img, patchSize=(
            int(rect['w']), int(rect['h'])), center=(int(rect['cx']), int(rect['cy'])))
        img_cropped = cv2.copyMakeBorder(
            img_cropped, top=100, bottom=100, left=100, right=100, borderType=cv2.BORDER_CONSTANT, value=(0, 0, 0))
        arrows.append(img_cropped)
        
    for arrow in arrows:
        edges = cv2.Canny(arrow, 200, 200)
        lines = cv2.HoughLinesP(
            edges, rho=1, theta=np.pi/180.0, threshold=100)
        if lines is None:
            continue
        temp = []
        for line in lines:
            x1, y1, x2, y2 = line[0]
            temp.append([y1, x1, x2, y2])
        temp.sort()
        y1, x1, x2, y2 = temp[0]
        ang = math.atan(float(y2-y1)/(x2-x1))
        if ang > np.pi/8 and ang < 3*np.pi/8:
            cv2.putText(img, "right", (100, 100), 0, 3, (0, 0, 255), 5)
        elif ang < -np.pi/8 and ang > -3*np.pi/8:
            cv2.putText(img, "left", (100, 100), 0, 3, (0, 0, 255), 5)
        else:
            pass
    return img


while True:
    _, img = capture.read()
    cv2.imshow("camera", img)
    img_arrow = detectArrow(img)
    cv2.imshow("result_arrow", img_arrow)
    input = cv2.waitKey(1)

/var/folders/85/1nl3clgd399drvmkl6hn19bm0000gn/T/ipykernel_95934/3101423606.py:79: RuntimeWarning: invalid value encountered in true_divide
  ang = math.atan(float(y2-y1)/(x2-x1))
/var/folders/85/1nl3clgd399drvmkl6hn19bm0000gn/T/ipykernel_95934/3101423606.py:79: RuntimeWarning: divide by zero encountered in true_divide
  ang = math.atan(float(y2-y1)/(x2-x1))


KeyboardInterrupt: 